In [7]:
import numpy as np
import pandas as pd
import pickle


# EEGNet-specific imports
from EEGModels import DeepConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix


In [8]:
def load_dataset_ctrvsone_and_transfer(user_split = True, runidx=1, data_type='meditation', one_class='HT'):
    X_train = None
    y_train_medid = None
    y_train_subjid = None
    X_val = None
    y_val_medid = None
    y_val_subjid = None
    X_test = None
    y_test_medid = None
    y_test_subjid = None

    dtype_names = ["HT", "SNY", "VIP", "CTR"]

    # user_split: Determines the creation of train/val/test set. In user_split, test/val users are never seen during the train. time_split randomly splits each user's chunks into train/val/test.
    if user_split:
        data_file_path = '../../iconip_data/{}/user_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)
    else:
        data_file_path = '../../iconip_data/{}/time_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)

    with open(data_file_path, 'rb') as f:
        all_data_splits = pickle.load(f)

        data_index_label_oneclass = dtype_names.index(one_class)
        data_index_label_ctr = dtype_names.index('CTR')
        transfer_index_labels = [x for x in list(set([0,1,2,3]).difference(set([dtype_names.index(one_class), 3]))) ]
        
        X_train = all_data_splits['train']['x']
        y_train_medid = all_data_splits['train']['y_med']
        y_train_subjid = all_data_splits['train']['y_subj']
        y_train_ts = all_data_splits['train']['y_ts']
        clf_config_subset_indices = np.where((y_train_medid == data_index_label_oneclass) | (y_train_medid == data_index_label_ctr))[0]
        X_train = X_train[(clf_config_subset_indices)]
        y_train_medid = y_train_medid[(clf_config_subset_indices)]
        y_train_subjid = y_train_subjid[clf_config_subset_indices]
        y_train_ts = y_train_ts[(clf_config_subset_indices)]

        X_val = all_data_splits['val']['x']
        y_val_medid = all_data_splits['val']['y_med']
        y_val_subjid = all_data_splits['val']['y_subj']
        y_val_ts = all_data_splits['val']['y_ts']
        clf_config_subset_indices = np.where((y_val_medid == data_index_label_oneclass) | (y_val_medid == data_index_label_ctr))[0]
        X_val = X_val[(clf_config_subset_indices)]
        y_val_medid = y_val_medid[(clf_config_subset_indices)]
        y_val_subjid = y_val_subjid[(clf_config_subset_indices)]
        y_val_ts = y_val_ts[(clf_config_subset_indices)]
        
        # Pretrain test and transfer test set
        X_test = all_data_splits['test']['x']
        y_test_medid = all_data_splits['test']['y_med']
        y_test_subjid = all_data_splits['test']['y_subj']
        y_test_ts = all_data_splits['test']['y_ts']
        
        # transfer test contains CTR from test and other two classes from the test.
        ctr_othertwo_indices = np.where((y_test_medid == data_index_label_ctr) | (y_test_medid == transfer_index_labels[0]) | (y_test_medid == transfer_index_labels[1]))[0]
        X_test_transfer = X_test[(ctr_othertwo_indices)]
        y_test_transfer_medid = y_test_medid[(ctr_othertwo_indices)]
        y_test_transfer_subjid = y_test_subjid[(ctr_othertwo_indices)]
        
        # pretrain test filter CTR and one_class from the test.
        clf_config_subset_indices = np.where((y_test_medid == data_index_label_oneclass) | (y_test_medid == data_index_label_ctr))[0]
        X_test = X_test[(clf_config_subset_indices)]
        y_test_medid = y_test_medid[(clf_config_subset_indices)]
        y_test_subjid = y_test_subjid[(clf_config_subset_indices)]
        y_test_ts = y_test_ts[(clf_config_subset_indices)]

    return X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid

In [9]:
def train_model(user_split, med_tech_clf, data_type, one_class):
    
    kernels, chans, samples = 1, 64, 2560
    
    split_strat = 'user' if user_split else 'time'
    model_results = pd.DataFrame()

    for runidx in range(1, 6):
        model_results['{}_Train_Acc'.format(runidx)] = None
        model_results['{}_Val_Acc'.format(runidx)] = None
        model_results['{}_Test_Acc'.format(runidx)] = None
        model_results['{}_Test_Transfer_Acc'.format(runidx)] = None
        model_results['{}_best_params'.format(runidx)] = None


    for runidx in range(1, 6):
        print("RUN ID: ", runidx)
        med_tech_clf = True
        
        X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_transfer, y_test_transfer_medid, y_test_transfer_subjid = load_dataset_ctrvsone_and_transfer(user_split=user_split, runidx=runidx, data_type=data_type, one_class=one_class)
        ## Convert data to binary classification task, i.e. meditation expert as class 0 vs control group as class 1
        y_train_medid = y_train_medid//3
        y_val_medid = y_val_medid//3
        y_test_medid = y_test_medid//3
        y_test_transfer_medid = y_test_transfer_medid//3

        best_clf_ours = None
        best_clf_val = 0

        #X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid
        if med_tech_clf:
            Y_train      = y_train_medid
            Y_validate   = y_val_medid
            Y_test       = y_test_medid
            Y_transfer   = y_test_transfer_medid
            ckpt_file_suffix = '{}_med_clf'.format(split_strat)
        else:
            Y_train      = y_train_subjid
            Y_validate   = y_val_subjid
            Y_test       = y_test_subjid
            Y_transfer   = y_test_transfer_subjid
            ckpt_file_suffix = '{}_subj_clf'.format(split_strat)
        nb_classes = np.unique(Y_train).shape[0]
        print(np.unique(Y_train), ": unique class labels.")
    
        ############################# EEGNet portion ##################################
        # convert labels to one-hot encodings.
        Y_train      = np_utils.to_categorical(Y_train)
        Y_validate   = np_utils.to_categorical(Y_validate)
        Y_test       = np_utils.to_categorical(Y_test)
        Y_transfer   = np_utils.to_categorical(Y_transfer)

        # convert data to NHWC (trials, channels, samples, kernels) format. Data 
        # contains 60 channels and 151 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
        X_val        = X_val.reshape(X_val.shape[0], chans, samples, kernels)
        X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
        X_transfer   = X_transfer.reshape(X_transfer.shape[0], chans, samples, kernels)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_test.shape[0], 'test samples')
        print(X_transfer.shape[0], 'transfer samples')
        print(nb_classes, " number of classes")
    
        # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
        # model configurations may do better, but this is a good starting point)
        model = DeepConvNet(nb_classes = nb_classes, Chans = chans, Samples = samples, 
                    dropoutRate = 0.5)

        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', optimizer='adam', 
                    metrics = ['accuracy'])

        # count number of parameters in the model
        numParams    = model.count_params()    

        # set a valid path for your system to record model checkpoints
        checkpointer = ModelCheckpoint(filepath='./ckpt/deepconv/checkpoint_{}.h5'.format(ckpt_file_suffix), 
                                    verbose=1, save_best_only=True)
        
        ###############################################################################
        # if the classification task was imbalanced (significantly more trials in one
        # class versus the others) you can assign a weight to each class during 
        # optimization to balance it out. This data is approximately balanced so we 
        # don't need to do this, but is shown here for illustration/completeness. 
        ###############################################################################

        # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
        # the weights all to be 1
        class_weights = {cl: 1 for cl in range(nb_classes)}

        ################################################################################
        # fit the model. Due to very small sample sizes this can get
        # pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
        # Riemannian geometry classification (below)
        ################################################################################
        escallback = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
        fittedModel = model.fit(X_train, Y_train, batch_size = 128, epochs = 15, 
                                verbose = 2, validation_data=(X_val, Y_validate),
                                callbacks=[checkpointer, escallback], class_weight = class_weights)

        # load optimal weights
        model.load_weights('./ckpt/deepconv/checkpoint_{}.h5'.format(ckpt_file_suffix))
    
        probs       = model.predict(X_test)
        preds       = probs.argmax(axis = -1)

        probs_transfer = model.predict(X_transfer)
        preds_transfer = probs_transfer.argmax(axis = -1)
        
        # save preds on train and val set for error analysis
        preds_train_eegnet = model.predict(X_train)
        preds_train_eegnet = preds_train_eegnet.argmax(axis = -1)
        preds_val_eegnet = model.predict(X_val)
        preds_val_eegnet = preds_val_eegnet.argmax(axis = -1)
        
        
        acc_test         = np.mean(preds == Y_test.argmax(axis=-1))
        acc_transfer     = np.mean(preds_transfer == Y_transfer.argmax(axis=-1))
        print("DeepConv Classification accuracy test and transfer: %f %f" % (acc_test, acc_transfer))
        model_results.loc["DeepConv", ['{}_Train_Acc'.format(runidx), '{}_Val_Acc'.format(runidx), '{}_Test_Acc'.format(runidx), '{}_Test_Transfer_Acc'.format(runidx)]] = [preds_train_eegnet, preds_val_eegnet, acc_test, acc_transfer]
    if user_split:
        model_results.to_csv('./RESULTS_T5{}_MED_TransferPRED_{}_CTRvs{}_DeepConv.csv'.format('USER', data_type, one_class))
    else:
        model_results.to_csv('./RESULTS_T5_MED_TransferPRED_{}_CTRvs{}_DeepConv.csv'.format(data_type, one_class))

    return model_results

In [10]:
# time split
for one_class in ['HT', 'SNY', 'VIP']:
    model_results = train_model(user_split=False, med_tech_clf=True, data_type='meditation', one_class=one_class)

RUN ID:  1
[0 1] : unique class labels.
X_train shape: (990, 64, 2560, 1)
990 train samples
344 test samples
523 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 1.45566, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 32s - loss: 3.4695 - accuracy: 0.4949 - val_loss: 1.4557 - val_accuracy: 0.5861 - 32s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss improved from 1.45566 to 0.82855, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 31s - loss: 1.7164 - accuracy: 0.5990 - val_loss: 0.8286 - val_accuracy: 0.5438 - 31s/epoch - 4s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.82855
8/8 - 31s - loss: 0.9840 - accuracy: 0.6687 - val_loss: 0.9565 - val_accuracy: 0.5106 - 31s/epoch - 4s/step
Epoch 4/15

Epoch 4: val_loss improved from 0.82855 to 0.51073, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 32s - loss: 0.6281 - accuracy: 0.7212 - val_loss: 0.5107 - val_accuracy: 0.7462 - 32s/epoch - 4

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[0 1] : unique class labels.
X_train shape: (990, 64, 2560, 1)
990 train samples
344 test samples
523 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 2.76821, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 44s - loss: 2.9952 - accuracy: 0.5434 - val_loss: 2.7682 - val_accuracy: 0.5468 - 44s/epoch - 6s/step
Epoch 2/15

Epoch 2: val_loss improved from 2.76821 to 0.58497, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 39s - loss: 1.4315 - accuracy: 0.5667 - val_loss: 0.5850 - val_accuracy: 0.6586 - 39s/epoch - 5s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.58497
8/8 - 36s - loss: 0.8904 - accuracy: 0.6515 - val_loss: 0.6582 - val_accuracy: 0.6012 - 36s/epoch - 5s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.58497
8/8 - 38s - loss: 0.7404 - accuracy: 0.6869 - val_loss: 0.9805 - val_accuracy: 0.4653 - 38s/epoch - 5s/step
Epoch 5/15

Epoch 5: val_loss improved from 0.58497 to 0.55656, savi

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (990, 64, 2560, 1)
990 train samples
344 test samples
523 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 1.56411, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 37s - loss: 2.6458 - accuracy: 0.5141 - val_loss: 1.5641 - val_accuracy: 0.5347 - 37s/epoch - 5s/step
Epoch 2/15

Epoch 2: val_loss improved from 1.56411 to 0.56899, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 39s - loss: 1.2985 - accuracy: 0.5434 - val_loss: 0.5690 - val_accuracy: 0.6647 - 39s/epoch - 5s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.56899
8/8 - 37s - loss: 1.0888 - accuracy: 0.6131 - val_loss: 0.8390 - val_accuracy: 0.6647 - 37s/epoch - 5s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.56899
8/8 - 39s - loss: 0.8471 - accuracy: 0.6778 - val_loss: 1.3655 - val_accuracy: 0.5045 - 39s/epoch - 5s/step
Epoch 5/15

Epoch 5: val_loss improved from 0.56899 to 0.43665, savi

...

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (996, 64, 2560, 1)
996 train samples
351 test samples
516 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 6.14466, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 35s - loss: 3.5993 - accuracy: 0.5743 - val_loss: 6.1447 - val_accuracy: 0.5331 - 35s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss improved from 6.14466 to 0.74823, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 35s - loss: 2.3781 - accuracy: 0.6637 - val_loss: 0.7482 - val_accuracy: 0.6476 - 35s/epoch - 4s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.74823
8/8 - 40s - loss: 0.9249 - accuracy: 0.7098 - val_loss: 1.1741 - val_accuracy: 0.6536 - 40s/epoch - 5s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.74823
8/8 - 37s - loss: 0.6544 - accuracy: 0.7279 - val_loss: 1.0289 - val_accuracy: 0.7048 - 37s/epoch - 5s/step
Epoch 5/15

Epoch 5: val_loss improved from 0.74823 to 0.70808, savi

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (996, 64, 2560, 1)
996 train samples
351 test samples
516 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 4.02079, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 41s - loss: 1.9309 - accuracy: 0.5753 - val_loss: 4.0208 - val_accuracy: 0.5994 - 41s/epoch - 5s/step
Epoch 2/15

Epoch 2: val_loss improved from 4.02079 to 0.99049, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 43s - loss: 1.0847 - accuracy: 0.6536 - val_loss: 0.9905 - val_accuracy: 0.6536 - 43s/epoch - 5s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.99049
8/8 - 43s - loss: 1.1300 - accuracy: 0.7149 - val_loss: 1.9135 - val_accuracy: 0.6235 - 43s/epoch - 5s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.99049
8/8 - 40s - loss: 0.7451 - accuracy: 0.7279 - val_loss: 1.9256 - val_accuracy: 0.6476 - 40s/epoch - 5s/step
Epoch 5/15

Epoch 5: val_loss improved from 0.99049 to 0.53872, savi

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [11]:
print("Done")

Done


In [5]:
# User split
for one_class in ['HT', 'SNY', 'VIP']:
    model_results = train_model(user_split=True, med_tech_clf=True, data_type='meditation', one_class=one_class)

RUN ID:  1
[0 1] : unique class labels.
X_train shape: (1038, 64, 2560, 1)
1038 train samples
308 test samples
447 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 2.00379, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 32s - loss: 3.2745 - accuracy: 0.5289 - val_loss: 2.0038 - val_accuracy: 0.7398 - 32s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss improved from 2.00379 to 1.24370, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 32s - loss: 1.6293 - accuracy: 0.5559 - val_loss: 1.2437 - val_accuracy: 0.7147 - 32s/epoch - 4s/step
Epoch 3/15

Epoch 3: val_loss improved from 1.24370 to 0.65254, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 29s - loss: 0.8427 - accuracy: 0.6358 - val_loss: 0.6525 - val_accuracy: 0.7492 - 29s/epoch - 3s/step
Epoch 4/15

Epoch 4: val_loss improved from 0.65254 to 0.55939, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 30s - loss: 0.5709 - accuracy

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[0 1] : unique class labels.
X_train shape: (1031, 64, 2560, 1)
1031 train samples
325 test samples
460 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 1.75091, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 33s - loss: 3.5044 - accuracy: 0.5092 - val_loss: 1.7509 - val_accuracy: 0.4660 - 33s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss improved from 1.75091 to 0.75920, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 30s - loss: 2.8372 - accuracy: 0.4908 - val_loss: 0.7592 - val_accuracy: 0.5890 - 30s/epoch - 3s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.75920
9/9 - 31s - loss: 1.6466 - accuracy: 0.5121 - val_loss: 4.0991 - val_accuracy: 0.5113 - 31s/epoch - 3s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.75920
9/9 - 30s - loss: 1.3122 - accuracy: 0.5276 - val_loss: 1.6621 - val_accuracy: 0.4757 - 30s/epoch - 3s/step
Epoch 5/15

Epoch 5: val_loss did not improve from 0.75920
9/9 - 2

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1058, 64, 2560, 1)
1058 train samples
311 test samples
485 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 3.96470, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 33s - loss: 2.4331 - accuracy: 0.5907 - val_loss: 3.9647 - val_accuracy: 0.5101 - 33s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss improved from 3.96470 to 3.06844, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 31s - loss: 1.2053 - accuracy: 0.6673 - val_loss: 3.0684 - val_accuracy: 0.6115 - 31s/epoch - 3s/step
Epoch 3/15

Epoch 3: val_loss improved from 3.06844 to 1.41675, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 29s - loss: 0.8560 - accuracy: 0.7845 - val_loss: 1.4167 - val_accuracy: 0.6014 - 29s/epoch - 3s/step
Epoch 4/15

Epoch 4: val_loss improved from 1.41675 to 0.68221, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 31s - loss: 0.4430 - accuracy: 0.8053 - 

...

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1042, 64, 2560, 1)
1042 train samples
315 test samples
461 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 1.20033, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 32s - loss: 1.3473 - accuracy: 0.6180 - val_loss: 1.2003 - val_accuracy: 0.5124 - 32s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss did not improve from 1.20033
9/9 - 29s - loss: 1.0233 - accuracy: 0.7217 - val_loss: 4.8965 - val_accuracy: 0.4845 - 29s/epoch - 3s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 1.20033
9/9 - 30s - loss: 0.5783 - accuracy: 0.7726 - val_loss: 3.8242 - val_accuracy: 0.4845 - 30s/epoch - 3s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 1.20033
9/9 - 30s - loss: 0.3850 - accuracy: 0.8733 - val_loss: 1.2836 - val_accuracy: 0.6273 - 30s/epoch - 3s/step
11/11 [==============================] - 2s 160ms/step
DeepConv Classification accuracy test and transfer: 0.682540 0.449024
RUN I

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1045, 64, 2560, 1)
1045 train samples
316 test samples
486 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 0.64740, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 32s - loss: 1.4564 - accuracy: 0.5598 - val_loss: 0.6474 - val_accuracy: 0.6635 - 32s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss did not improve from 0.64740
9/9 - 29s - loss: 0.7512 - accuracy: 0.6029 - val_loss: 0.6745 - val_accuracy: 0.7799 - 29s/epoch - 3s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.64740
9/9 - 30s - loss: 0.7554 - accuracy: 0.6756 - val_loss: 2.2797 - val_accuracy: 0.5283 - 30s/epoch - 3s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.64740
9/9 - 31s - loss: 0.5922 - accuracy: 0.7388 - val_loss: 1.6468 - val_accuracy: 0.7830 - 31s/epoch - 3s/step
10/10 [==============================] - 2s 219ms/step
DeepConv Classification accuracy test and transfer: 0.553797 0.565844


/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [6]:
print("Done")

Done
